In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Práctico 3 - Introducción al Aprendizaje Automático

## Modelos de Precios de Mercado
Queremos crear un modelo que prediga si el precio de una acción va a subir o bajar de acuerdo a información del pasado. Para ello implementaremos algunos modelos derivados de la teoria económica respecto de como valuar una activo con flujo de fondos inciertos.


### Capital Asset Pricing Model (CAPM)

El modelo de CAPM propuesto por William Sharpe a partir del desarrollo de Markowitz sobre carteras eficientes, busca predecir el retorno de un activo en función del comportamiento global del mercado, y de la relación del activo puntual con el mercado. 

Según este modelo, que se puede expresar de la siguiente manera: 

$$
E(r_i) = r_f + \beta  [ E(r_m) - r_f ] 
$$

El retorno esperado del activo i, es igual al retorno libre de riesgo (o tasa libre de riesgo), más el exceso de retorno esperado de mercado (es decir lo que se espera que rinda el mercado por sobre la tasa libre de riesgo) escalado por un coeficiente beta. Este coeficiente beta es el conciente entre la covarianza del activo con el mercado sobre la varianza del mercado, y se interpreta como un multiplicador de riesgo de mercado.

Pueden ver una explicación completa en: https://economipedia.com/definiciones/modelo-valoracion-activos-financieros-capm.html

Adicionalmente, se puede agregar un término independiente como ordenada al origen alfa, que represanta una ganancia extraordinaria no explicada por la relacion con de la acción con el mercado sino por factores extraordinarios o arbitrages en caso de estrategias. De allí el famoso "buscar alfa" siendo una directriz de todos los portafolios managers. 

$$
E(r_i) = \alpha + r_f + \beta  [ E(r_m) - r_f ]
$$

Para realizar este ejercicio, deberán descargar el precio de una acción y del mercado asociado (por ejemplo GOOG y NASDAQ). En ambos casos tomar los retornos y tomar una tasa libre de riesgo. A continuación, deberán probar varias regresiones para comprobar el beta, y examinar si el agregado de una ordenada al origen aporta a los resultados. La idea es que seleccionen varias acciones para modelarlo con diferentes activos. 

Una vez que lo hayan hecho, adionalmente responder las siguientes preguntas:

1. ¿Es fácil encontrar $\alpha$ distinto de cero?

2. ¿Qué interpretación le podemos dar a $\beta$?

3. ¿En qué momento nos gustaría buscar activos con $\beta$ > 1 y $\beta$ < 1? ¿Qué implica tener un $\beta$ < 0?



In [16]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
data_1 = pdr.get_data_yahoo("NDAQ GOOG", start="2000-01-01", end="2021-07-31")
data_2 = pdr.get_data_yahoo("KO NYA", start="2000-01-01", end="2021-07-31")

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed


In [17]:
data_respaldo_1 = data_1
data_respaldo_2 = data_2

# Reseteo MultiIndex y renombro columnas
data_1 = data_1.reset_index(level='Date') # Reseteo MultiIndex (Date en este caso y la guardaré como columna)
data_2 = data_2.reset_index(level='Date')

In [18]:
data_1.head()

Date Adj Close            Close        High        Low       Open  \
                  GOOG       NDAQ  GOOG   NDAQ GOOG  NDAQ GOOG  NDAQ GOOG   
0 2002-07-01       NaN  12.815305   NaN  15.00  NaN  15.0  NaN  15.0  NaN   
1 2002-07-02       NaN  12.926373   NaN  15.13  NaN  16.5  NaN  15.0  NaN   
2 2002-07-03       NaN  12.815305   NaN  15.00  NaN  16.0  NaN  14.0  NaN   
3 2002-07-05       NaN  12.815305   NaN  15.00  NaN  15.0  NaN  15.0  NaN   
4 2002-07-08       NaN  11.960951   NaN  14.00  NaN  14.0  NaN  14.0  NaN   

        Volume         
   NDAQ   GOOG   NDAQ  
0  15.0    NaN   1300  
1  15.5    NaN   7000  
2  14.0    NaN  10200  
3  15.0    NaN    500  
4  14.0    NaN    300

In [5]:
# Retornos
#variation_close_nd = data_nd.Close.pct_change()*100 
#variation_close_go = data_go.Close.pct_change()*100
#variation_close_ko = data_ko.Close.pct_change()*100
#variation_close_ny = data_ny.Close.pct_change()*100

## consultar si se hace el pct change sobre close o adj close

In [24]:
#  Calculamos los retornos logarítmicos y sacamos la covarianza anualizada entre los dos.
log_returns_1 = np.log(1 + data_1.Close.pct_change())
cov_1 = log_returns_1.cov()*252 # 252 dias habiles del año
cov_1

,GOOG,NDAQ
GOOG,0.091310,0.048305
NDAQ,0.048305,0.154627


In [23]:
# Fijamos el dato 0,048305 con iloc[0,1], sacamos la varianza .var() anualizada del NDAQ y con esto tendremos las Betas.
cov_with_market_1 = cov_1.iloc[0,1]
market_var_1 = log_returns_1['NDAQ'].var()*252
stock_beta_1 = cov_with_market_1 / market_var_1
stock_beta_1

0.3123950394067975

In [27]:
# Definimos un riskfree, siendo este el rendimiento del bono a 10 años estadounidense, a precio de cierre de ayer de 0,95%

# Para el riskpremium calculamos la fórmula: (Rm - Rf) anualizando la media de los retornos del NDAQ menos el riskfree.

riskfree_1 = 0.0095
riskpremium_1 = (log_returns_1['NDAQ'].mean()*252) - riskfree_1

In [28]:
# Generamos la variable de la fórmula del CAPM, stock_capm_return, y otra para el ratio Sharpe, 
# que mide el grado de rentabilidad ajustado por el riesgo, cuanto mayor sea éste mejor.

stock_capm_return_1 = riskfree_1 + stock_beta_1 * riskpremium_1
sharpe_stock_1 = (stock_capm_return_1 - riskfree_1) /(log_returns_1['GOOG'].std()*252**0.5)

In [29]:
tickers = ['GOOG', 'NDAQ']
print("La Beta de " + str(tickers) + " es de: " + str(round(stock_beta_1,3)))
print("El retorno CAPM de " + str(tickers) + " es de: " + str(round(stock_capm_return_1*100,3))+"%")
print("El Ratio Sharpe de " + str(tickers) + " es de: " + str(round(sharpe_stock_1,3)))

La Beta de ['GOOG', 'NDAQ'] es de: 0.312
El retorno CAPM de ['GOOG', 'NDAQ'] es de: 4.787%
El Ratio Sharpe de ['GOOG', 'NDAQ'] es de: 0.127


- ¿Es fácil encontrar α distinto de cero?

- ¿Qué interpretación le podemos dar a β ?

Beta mide el riesgo sistematico, aquel que viene dado por fluctuaciones del mercado, es decir, por las variables macroeconómicas, y que es inherente al mercado, siendo éste incapaz de eliminarse por completo.

- ¿En qué momento nos gustaría buscar activos con β > 1 y β < 1? ¿Qué implica tener un β < 0?

Beta superior a uno significa que el activo o cartera se mueve en mayor proporción que el benchmark, tanto al alza como a la baja.

Si la beta se encuentra entre cero y uno, se trataría de un activo defensivo ya que es menos arriesgado que el mercado. Cuando el mercado sube, el activo lo hace con menor intensidad, y viceversa.

Que la beta sea igual a cero quiere decir que la covarianza del activo con respecto al mercado es cero. Esto ocurriría con el activo libre de riesgo, por ejemplo. 

### Modelos Autorregresivos para predecir precios.

Para esta parte del práctico, nos gustaría utilizar precios del pasado para poder estimar cuál va a ser el precio de una acción más adelante. Las regresiones que vemos en la introducción al aprendizaje automático no son las mismas que deben usarse cuando estamos trabajando con precios de mercado, ya que estos datos son series de tiempo. Para trabajar con este tipo de datos, es necesario generar **modelos autorregresivos**.

Si suponemos que $y_t$ es el precio de nuestra acción en el tiempo $t$, podemos generar el siguiente modelo autorregresivo:

$$
y_t = \delta + \phi_1 y_{t-1} + \dots + \phi_p y_{t-p} + \epsilon_t
$$

Existen otras variables que podemos agregar (como estacionariedad o tendencias), pero para este práctico podemos usar solamente el modelo `AutoReg` de la librería StatsModels: https://www.statsmodels.org/stable/examples/notebooks/generated/autoregressions.html.

El **ejercicio** para esta sección es crear una función que reciba 4 parámetros:
- El *ticker* de un activo.
- Una *fecha* a partir de la cual se van a querer predecir los precios del activo.
- Un *número de días* hacia atrás que se van a usar para ajustar el modelo autorregresivo.
- Un *número de días* hacia adelante para los cuales van a querer predecir el precio del activo utilizando el modelo entrenado.

Esta función se debe encargar de buscar los precios del ticker, realizar el ajuste y predecir el precio de la acción hacia delante. Además, debe devolver el [error cuadrático medio](https://es.wikipedia.org/wiki/Error_cuadr%C3%A1tico_medio) de la predicción, comparado con los verdaderos precios del activo.

¿Qué pasa si tratamos de predecir el precio del SPY el día antes del primer cisne negro de 2020 con un modelo que use 15 días hacia atrás?

### Árboles de Decisión para predecir suba o baja.

Venimos trabajando los precios con valores continuos, pero otra forma de verlos podría ser convirtiendolos en, por ejemplo, -1 y 1 para saber si el precio está en suba o en baja. De esta forma obtenemos un problema de clasificación que nos permite mirar los datos desde otra perspectiva.

- Etiqueten los datos con 1 si [pct_change](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) es mayor a cero y 0, -1 si no.
- Entrenen un árbol de decisión que prediga si el valor de un activo va a subir o bajar.
- Reporten las siguientes métricas sobre el modelo resultante.

    - Accuracy
    - Precision
    - Recall
    - F1
    - matriz de confusión

El etiquetado de datos no suele ser trivial dado a que depende de la naturaleza de los datos. Posiblemente, el precio de un activo suba un poco pero luego continue su declive ¿Cuánto tiene que subir/bajar un precio para ser considerado "suba"/"baja"?. ¿Tiene sentido tomar "periodos de suba/baja" más grandes para predecir si el comportamiento del activo en el futuro?. ¿Se podría utilizar un enfoque más estadístico para elegir el etiquetado?.

Aplicar GridSearch sobre los parámetros y luego para la mejor configuración encontrada, evaluar sobre el conjunto de entrenamiento y sobre el conjunto de evaluación, reportando:

- Accuracy
- Precision
- Recall
- F1
- matriz de confusión